In [ ]:
!pip install networkx

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def gen_random_A(num_nodes, edge_probability, directed):
    '''
    Generates a random adjacency matrix for a graph with no self-loops based on number of nodes, edge probability, and type of graph
    Parameters:
     - num_nodes: the number of nodes in the graph
     - edge_probability: probability of an edge between any two nodes
     - directed: whether or not the graph is directed
    Returns:
     - random_adjacency: the adjacency matrix representing the random graph
    '''
    random_adjacency = np.random.choice([0,1], size=[num_nodes,num_nodes], p=[(1-edge_probability),edge_probability]) 
    np.fill_diagonal(random_adjacency, 0) # Remove self-loops
    if(not directed):
        random_adjacency = np.triu(random_adjacency) | np.triu(random_adjacency, 1).T # Force the matrix to be symmetric (undirected)
    return random_adjacency

# Discussion 0: NetworkX Tutorial
[NetworkX Documentation](https://networkx.org/documentation/stable/reference/index.html)

## Creating a Graph from an adjacency matrix
 - Important step in real-world applications, when you might be **constructing** a graph from some tabular data.
 - Here, we'll generate a random adjacency matrix (exact steps aren't important) represent it as a graph.
 - Graphs can be represented as nx.Graph or nx.DiGraph
 - We can also get a quick visualization for any graph with nx.draw

In [ ]:
num_nodes = 15
edge_probability = 0.15
directed = True

# Generate a random, binary nxn matrix, with probability of a "1" denoted by edge_probability
random_adjacency = gen_random_A(num_nodes, edge_probability, directed)
if(not directed):
    G_r = nx.Graph(incoming_graph_data=random_adjacency)
if(directed):
    G_r = nx.DiGraph(incoming_graph_data=random_adjacency)

fig, ax = plt.subplots()
ax.set_title('Naive Random Graph')
nx.draw(G_r, ax=ax, with_labels=True)
display(random_adjacency, fig)
plt.close()

### Accessing Basic parts of a Graph

In [ ]:
display(type(G_r.nodes), G_r.nodes) # Graph Nodes
display(type(G_r.edges), G_r.edges) # Graph Edges
display(f'A single graph node: {G_r.nodes[0]}')
display(f'That node\'s neighbors: {[neighbor for neighbor in G_r.neighbors(0)]}')
display(f'A single graph edge: {G_r.edges[list(G_r.edges())[0]]}')

We can add node or edge attributes, which we think of as data over the graph.

In [ ]:
for n in G_r:
    G_r.nodes[n]['node_data'] = np.random.randn(3) # Associate with each node a 3-dimensional random vector
for u, v in G_r.edges():
    G_r.edges[u, v]['edge_data'] = G_r.nodes[u]['node_data'] + G_r.nodes[v]['node_data'] # Give each edge the sum of its nodes' data
display(f'A single graph node: {G_r.nodes[0]}')
display(f'A single graph edge: {G_r.edges[list(G_r.edges())[0]]}')

## Degrees & Shortest Paths
Here, we're going to switch from our trivial random graph to one that actually has some structure. The **Zachary's Karate Club** graph is a well-known toy dataset used for demonstrating many network science techniques. It documents the social connections between members of a university's Karate Club after a conflict between two instructors - the Officer and Mr. Hi. We can load it in directly via NetworkX.

In [ ]:
G = nx.karate_club_graph()
pos = nx.spring_layout(G, seed=42)
fig, ax = plt.subplots()
nx.draw(G, pos=pos, labels={n:n+1 for n in G}, node_color=[{'Mr. Hi':0,'Officer':1}[G.nodes[n]['club']] for n in G], 
        cmap=plt.cm.Set2, with_labels=True, ax=ax)
ax.set_title('Zachary\'s Karate Club');

**Question:** Which nodes represent the two instructors?

### Degree Distribution
Let's plot the degree distribution of this graph.

In [ ]:
degrees = G.degree()
display(degrees)

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(12,6))
plot = nx.draw(G, pos=pos, labels={n:n for n in G}, node_color=[dict(degrees)[n] for n in G], 
        cmap=plt.cm.viridis, with_labels=True, ax=axes[0]); # Draw the graph
axes[0].set_title('Zachary\'s Karate Club, colored by degree') # Set graph visualization title
degree_list = [dict(degrees)[n] for n in G] # Converge node degrees to a list
axes[1].hist(degree_list, edgecolor='black') # Plot histogram
axes[1].set_title('Degree distribution histogram'); # Set histogram title

### Calculating Path Lengths

In [ ]:
display(f'Shortest path length from the Officer to Mr. Hi: {nx.shortest_path_length(G, 0, 33)}')

In [ ]:
display(f'Shortest path from the Officer to Mr. Hi: {nx.shortest_path(G, 0, 33)}')

In [ ]:
display(f'All shortest paths from the Officer to Mr. Hi: {[path for path in nx.all_shortest_paths(G, 0, 33)]}')

In [ ]:
display(f'Average shortest path length across all pairs: {nx.average_shortest_path_length(G)}')

Also:
 - all pairs, all shortest paths
 - single source, all shortest paths
 - has path?

## Centrality Metrics

In [ ]:
# G = nx.lollipop_graph(m=8, n=8) # Fully-connected graph of size m connected to tail of length n
# pos = nx.spring_layout(G, seed=42)
# nx.draw(G)

In [ ]:
centrality = 'betweenness' # Choice of centrality metric
if(centrality == 'degree'):
    func = nx.degree_centrality
elif(centrality == 'betweenness'):
    func = nx.betweenness_centrality
elif(centrality == 'eigenvector'):
    func = nx.eigenvector_centrality
centralities = func(G)

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(12,6))
plot = nx.draw(G, pos=pos, labels={n:n+1 for n in G}, node_color=[dict(centralities)[n] for n in G], 
        cmap=plt.cm.viridis, with_labels=True, ax=axes[0]); # Draw the graph
axes[0].set_title(f'Graph colored by {centrality} centrality') # Set graph visualization title
degree_list = [dict(centralities)[n] for n in G] # Convert node centralities to list
axes[1].hist(degree_list, edgecolor='black') # Plot histogram
axes[1].set_title(f'{centrality} centrality distribution histogram') # Set histogram title